# Testing Fink filters

First download some ZTF data using the Fink Data Transfer service (https://fink-portal.org/download). Then load the data using Spark:

In [1]:
df = spark.read.format('parquet').load('ftransfer_ztf_2023-03-21_623768')

For example, downloading data from 2023/03/17, I obtain nearly 200,000 alerts:

In [2]:
df.count()

190920

## Import and apply the filter

In [3]:
import pyspark.sql.functions as F
from myfilter import example_filter

In [4]:
df_filtered = df.withColumn('flag', example_filter('cdsxmatch', 'magpsf')).filter(F.col('flag'))
df_filtered.count()

452

Great our filter keeps 452 objects out of 200,000! you can then inspect them easily using Spark or even Pandas:

In [5]:
pdf = df_filtered.toPandas()

23/03/21 17:46:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
pdf.groupby('finkclass').count().sort_values('objectId', ascending=False)['objectId']

finkclass
QSO                401
Seyfert_1           21
Star                 8
Blue                 5
RRLyr                3
AGN                  2
Radio                2
SN                   2
**                   1
BClG                 1
CataclyV*            1
EB*                  1
LP*_Candidate        1
QSO_Candidate        1
RRLyr_Candidate      1
X                    1
Name: objectId, dtype: int64

OK our filter is mostly targeting QSO (faint objects in SIMBAD)